<a href="https://colab.research.google.com/github/ForrestOfFidum/Automated-Offers/blob/master/Popertyware.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install requests pandas --quiet

import csv
import requests
import pandas as pd
from typing import Dict, Any, List, Optional
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


In [76]:
from google.colab import userdata

# Base URL from Propertyware spec
BASE_URL ="https://api.propertyware.com/pw/api/rest/v1"
BASE_URL = BASE_URL.rstrip("/")

CLIENT_ID = userdata.get("PROPERTYWARE_CLIENT_ID")
CLIENT_SECRET = userdata.get("PROPERTYWARE_CLIENT_SECRET")
SYSTEM_ID = userdata.get("PROPERTYWARE_SYSTEM_ID")

if not (CLIENT_ID and CLIENT_SECRET and SYSTEM_ID):
    raise ValueError(
        "Missing Colab secrets. Please add:\n"
        "  PROPERTYWARE_CLIENT_ID\n"
        "  PROPERTYWARE_CLIENT_SECRET\n"
        "  PROPERTYWARE_SYSTEM_ID\n"
        "in the Colab Secrets panel (left sidebar key icon)."
    )

HEADERS = {
    "x-propertyware-client-id": CLIENT_ID,
    "x-propertyware-client-secret": CLIENT_SECRET,   # <— using your CLIENT_SECRET here
    "x-propertyware-system-id": SYSTEM_ID,
    "accept": "application/json",
    "content-type": "application/json",
}

print("✅ Propertyware secrets loaded. Base URL:", BASE_URL)


✅ Propertyware secrets loaded. Base URL: https://api.propertyware.com/pw/api/rest/v1


In [77]:
COLUMNS = [
    "Property name *",
    "Unit number *",
    "Sub type *",
    "Operating bank account *",
    "Property reserve amount (optional)",
    "Property address line 1 *",
    "Property address line 2 (optional)",
    "Property address line 3 (optional)",
    "Unit address line 1 *",
    "Unit address line 2 (optional)",
    "Unit address line 3 (optional)",
    "City/Locality *",
    "State/Province/Territory *",
    "Postal code *",
    "Building description (Used for listings) (optional)",
    "Unit description (Used for listings) (optional)",
    "Size (optional)",
    "Bedrooms (optional)",
    "Bathrooms (optional)",
    "Market rent (optional)",
    "Tenant First Name *",
    "Tenant Last Name *",
    "Lease Start Date (optional)",
    "Lease End Date (optional)",
    "Rent Frequency (optional)",
    " Rent Amount ",
    "Rent Next Due Date",
    "Rent Memo",
    " Rent 2 Amount ",
    "Rent 2 Account",
    "Rent 2 Next Due Date",
    "Rent 2 Memo",
    " Rent 3 Amount ",
    "Rent 3 Account",
    "Rent 3 Next Due Date",
    "Rent 3 Memo",
    "Login email (optional)",
    "Alternate email (optional)",
    "Tenant Street Address Line 1 (optional)",
    "Tenant Street Address Line 2 (optional)",
    "Tenant Street Address Line 3 (optional)",
    "City/Locality (optional)",
    "State/Province/Territory (optional)",
    "Postal Code (optional)",
    "Tenant Home Phone (optional)",
    "Tenant Work Phone (optional)",
    "Tenant Mobile (optional)",
    "Tenant Fax (optional)",
    "Emergency Contact Name (optional)",
    "Emergency Contact Phone (optional)",
    "Date Of Birth (optional)",
    "Comments (optional)",
    "Rental owner name *",
    "Rental Owner Street Address Line 1 (optional)",
    "Rental Owner Street Address Line 2 (optional)",
    "Rental Owner Street Address Line 3 (optional)",
    "Rental Owner City/Locality (optional)",
    "Rental Owner State/Province/Territory (optional)",
    "Postal code (optional)",
    "Rental Owner Login email (optional)",
    "Alternate email (optional)",
    "Tax payer ID (TIN) (optional)",
    "Eligible for 1099 (optional)",
    "Rental Owner Home phone (optional)",
    "Rental Owner Work phone (optional)",
    "Rental Owner Mobile (optional)",
    "Rental Owner Fax (optional)",
    "Rental Owner Date of birth (optional)",
    "Comments (optional)",
    "Rental Owner Agreement start date (optional)",
    "Rental Owner Agreement end date (optional)",
]


In [78]:
def make_session() -> requests.Session:
    s = requests.Session()
    s.headers.update(HEADERS)

    retry = Retry(
        total=5,
        backoff_factor=1.5,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=frozenset(["GET", "POST", "PUT", "PATCH", "DELETE"]),
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retry)
    s.mount("https://", adapter)
    s.mount("http://", adapter)
    return s

SESSION = make_session()

def get(path: str, params: Optional[Dict[str, Any]] = None) -> requests.Response:
    url = f"{BASE_URL}{path}"
    r = SESSION.get(url, params=params, timeout=60)
    if r.status_code >= 400:
        raise RuntimeError(f"GET {r.url} -> {r.status_code}\n{r.text[:2000]}")
    return r

def paginate(path: str, limit: int = 500, extra_params: Optional[Dict[str, Any]] = None):
    offset = 0
    while True:
        params = {"limit": limit, "offset": offset}
        if extra_params:
            params.update(extra_params)

        r = get(path, params=params)
        data = r.json()

        if not data:
            break

        for item in data:
            yield item

        total = r.headers.get("X-Total-Count")
        if total is None:
            if len(data) < limit:
                break
        else:
            total = int(total)
            offset += limit
            if offset >= total:
                break


In [94]:
def fetch_buildings() -> List[Dict[str, Any]]:
    # ADJUST HERE if your tenant uses /properties instead of /buildings
    return list(paginate("/buildings"))

def fetch_units_for_building(building_id: str) -> List[Dict[str, Any]]:
    # ADJUST HERE to match your tenant
    try:
        return list(paginate(f"/buildings/{building_id}/units"))
    except RuntimeError:
        try:
            return list(paginate("/units", extra_params={"buildingId": building_id}))
        except RuntimeError:
            return []

def fetch_leases_for_unit(unit_id: str) -> List[Dict[str, Any]]:
    # ADJUST HERE
    try:
        return list(paginate(f"/units/{unit_id}/leases"))
    except RuntimeError:
        try:
            return list(paginate("/leases", extra_params={"unitId": unit_id}))
        except RuntimeError:
            return []

def fetch_tenants_for_lease(lease_id: str) -> List[Dict[str, Any]]:
    # ADJUST HERE
    try:
        return list(paginate(f"/leases/{lease_id}/tenants"))
    except RuntimeError:
        try:
            return list(paginate("/tenants", extra_params={"leaseId": lease_id}))
        except RuntimeError:
            return []

def fetch_owners_for_building(building_id: str) -> List[Dict[str, Any]]:
    # ADJUST HERE
    try:
        return list(paginate(f"/buildings/{building_id}/owners"))
    except RuntimeError:
        try:
            return list(paginate("/owners", extra_params={"buildingId": building_id}))
        except RuntimeError:
            return []

def fetch_bank_accounts() -> List[Dict[str, Any]]:
    for candidate in ("/bankAccounts", "/accounts"):
        try:
            return list(paginate(candidate))
        except RuntimeError:
            continue
    return []

In [95]:
def pick(d: Dict[str, Any], *keys, default=None):
    for k in keys:
        if k in d and d[k] is not None:
            return d[k]
    return default

def as_str(x):
    return "" if x is None else str(x)

def join_name(person: Dict[str, Any]) -> str:
    first = pick(person, "firstName", "first_name", default="")
    last  = pick(person, "lastName", "last_name", default="")
    return (first + " " + last).strip()


In [97]:
def map_row(building, unit, lease, tenant, owner, bank_accounts_by_id):

    # --- Building / Property ---
    property_name = pick(building, "name", "propertyName", "displayName", default="")
    sub_type = pick(building, "subType", "sub_type", "propertyType", default="")
    reserve_amt = pick(building, "reserveAmount", "reserve_amount")

    b_addr1 = pick(building, "addressLine1", "address1", "street1", default="")
    b_addr2 = pick(building, "addressLine2", "address2", "street2")
    b_addr3 = pick(building, "addressLine3", "address3", "street3")
    city = pick(building, "city", "locality", default="")
    state = pick(building, "state", "province", default="")
    postal = pick(building, "postalCode", "zip", "postal_code", default="")
    building_desc = pick(building, "description", "listingDescription", "publicDescription")

    operating_account_name = ""
    operating_account_id = pick(building, "operatingBankAccountId", "operating_account_id")
    if operating_account_id and operating_account_id in bank_accounts_by_id:
        operating_account_name = pick(bank_accounts_by_id[operating_account_id], "name", "accountName", default="")
    else:
        operating_account_name = pick(building, "operatingBankAccountName", "operatingBankAccount", default="")

    # --- Unit ---
    unit_number = pick(unit, "unitNumber", "number", "name", default="")
    u_addr1 = pick(unit, "addressLine1", "address1", "street1", default=b_addr1)
    u_addr2 = pick(unit, "addressLine2", "address2", "street2")
    u_addr3 = pick(unit, "addressLine3", "address3", "street3")
    unit_desc = pick(unit, "description", "listingDescription", "publicDescription")
    size = pick(unit, "size", "squareFeet", "sqft")
    beds = pick(unit, "bedrooms", "beds")
    baths = pick(unit, "bathrooms", "baths")
    market_rent = pick(unit, "marketRent", "market_rent", "rent")

    # --- Lease / Rent ---
    lease = lease or {}
    lease_start = pick(lease, "startDate", "leaseStartDate")
    lease_end = pick(lease, "endDate", "leaseEndDate")
    rent_freq = pick(lease, "rentFrequency", "frequency")
    rent_amt = pick(lease, "rentAmount", "rent")
    rent_next_due = pick(lease, "nextDueDate", "rentNextDueDate")
    rent_memo = pick(lease, "rentMemo", "memo")

    rent2_amt = pick(lease, "rent2Amount", "rentTwoAmount")
    rent2_acct = pick(lease, "rent2Account", "rentTwoAccount")
    rent2_next = pick(lease, "rent2NextDueDate", "rentTwoNextDueDate")
    rent2_memo = pick(lease, "rent2Memo", "rentTwoMemo")

    rent3_amt = pick(lease, "rent3Amount", "rentThreeAmount")
    rent3_acct = pick(lease, "rent3Account", "rentThreeAccount")
    rent3_next = pick(lease, "rent3NextDueDate", "rentThreeNextDueDate")
    rent3_memo = pick(lease, "rent3Memo", "rentThreeMemo")

    # --- Tenant ---
    tenant = tenant or {}
    tenant_first = pick(tenant, "firstName", "first_name", default="")
    tenant_last = pick(tenant, "lastName", "last_name", default="")
    login_email = pick(tenant, "email", "loginEmail")
    alt_email = pick(tenant, "alternateEmail", "altEmail")

    t_addr1 = pick(tenant, "addressLine1", "address1", "street1")
    t_addr2 = pick(tenant, "addressLine2", "address2", "street2")
    t_addr3 = pick(tenant, "addressLine3", "address3", "street3")
    t_city = pick(tenant, "city")
    t_state = pick(tenant, "state")
    t_postal = pick(tenant, "postalCode", "zip")

    t_home = pick(tenant, "homePhone")
    t_work = pick(tenant, "workPhone")
    t_mobile = pick(tenant, "mobilePhone", "cellPhone")
    t_fax = pick(tenant, "fax")

    emerg_name = pick(tenant, "emergencyContactName")
    emerg_phone = pick(tenant, "emergencyContactPhone")
    dob = pick(tenant, "dateOfBirth", "dob")
    tenant_comments = pick(tenant, "comments", "notes")

    # --- Owner ---
    owner = owner or {}
    owner_name = join_name(owner) if owner else ""

    o_addr1 = pick(owner, "addressLine1", "address1", "street1")
    o_addr2 = pick(owner, "addressLine2", "address2", "street2")
    o_addr3 = pick(owner, "addressLine3", "address3", "street3")
    o_city = pick(owner, "city")
    o_state = pick(owner, "state")
    o_postal = pick(owner, "postalCode", "zip")

    o_login_email = pick(owner, "email", "loginEmail")
    o_alt_email = pick(owner, "alternateEmail", "altEmail")

    tin = pick(owner, "tin", "taxPayerId", "taxId")
    eligible_1099 = pick(owner, "eligibleFor1099", "eligible_1099")

    o_home = pick(owner, "homePhone")
    o_work = pick(owner, "workPhone")
    o_mobile = pick(owner, "mobilePhone", "cellPhone")
    o_fax = pick(owner, "fax")
    o_dob = pick(owner, "dateOfBirth", "dob")
    owner_comments = pick(owner, "comments", "notes")

    owner_agreement_start = pick(owner, "agreementStartDate")
    owner_agreement_end = pick(owner, "agreementEndDate")

    return {
        "Property name *": property_name,
        "Unit number *": unit_number,
        "Sub type *": sub_type,
        "Operating bank account *": operating_account_name,
        "Property reserve amount (optional)": as_str(reserve_amt),

        "Property address line 1 *": as_str(b_addr1),
        "Property address line 2 (optional)": as_str(b_addr2),
        "Property address line 3 (optional)": as_str(b_addr3),

        "Unit address line 1 *": as_str(u_addr1),
        "Unit address line 2 (optional)": as_str(u_addr2),
        "Unit address line 3 (optional)": as_str(u_addr3),

        "City/Locality *": city,
        "State/Province/Territory *": state,
        "Postal code *": postal,

        "Building description (Used for listings) (optional)": as_str(building_desc),
        "Unit description (Used for listings) (optional)": as_str(unit_desc),

        "Size (optional)": as_str(size),
        "Bedrooms (optional)": as_str(beds),
        "Bathrooms (optional)": as_str(baths),
        "Market rent (optional)": as_str(market_rent),

        "Tenant First Name *": tenant_first,
        "Tenant Last Name *": tenant_last,
        "Lease Start Date (optional)": as_str(lease_start),
        "Lease End Date (optional)": as_str(lease_end),
        "Rent Frequency (optional)": as_str(rent_freq),

        " Rent Amount ": as_str(rent_amt),
        "Rent Next Due Date": as_str(rent_next_due),
        "Rent Memo": as_str(rent_memo),

        " Rent 2 Amount ": as_str(rent2_amt),
        "Rent 2 Account": as_str(rent2_acct),
        "Rent 2 Next Due Date": as_str(rent2_next),
        "Rent 2 Memo": as_str(rent2_memo),

        " Rent 3 Amount ": as_str(rent3_amt),
        "Rent 3 Account": as_str(rent3_acct),
        "Rent 3 Next Due Date": as_str(rent3_next),
        "Rent 3 Memo": as_str(rent3_memo),

        "Login email (optional)": as_str(login_email),
        "Alternate email (optional)": as_str(alt_email),

        "Tenant Street Address Line 1 (optional)": as_str(t_addr1),
        "Tenant Street Address Line 2 (optional)": as_str(t_addr2),
        "Tenant Street Address Line 3 (optional)": as_str(t_addr3),
        "City/Locality (optional)": as_str(t_city),
        "State/Province/Territory (optional)": as_str(t_state),
        "Postal Code (optional)": as_str(t_postal),

        "Tenant Home Phone (optional)": as_str(t_home),
        "Tenant Work Phone (optional)": as_str(t_work),
        "Tenant Mobile (optional)": as_str(t_mobile),
        "Tenant Fax (optional)": as_str(t_fax),
        "Emergency Contact Name (optional)": as_str(emerg_name),
        "Emergency Contact Phone (optional)": as_str(emerg_phone),
        "Date Of Birth (optional)": as_str(dob),
        "Comments (optional)": as_str(tenant_comments),

        "Rental owner name *": owner_name,
        "Rental Owner Street Address Line 1 (optional)": as_str(o_addr1),
        "Rental Owner Street Address Line 2 (optional)": as_str(o_addr2),
        "Rental Owner Street Address Line 3 (optional)": as_str(o_addr3),
        "Rental Owner City/Locality (optional)": as_str(o_city),
        "Rental Owner State/Province/Territory (optional)": as_str(o_state),
        "Postal code (optional)": as_str(o_postal),
        "Rental Owner Login email (optional)": as_str(o_login_email),
        "Alternate email (optional)": as_str(o_alt_email),
        "Tax payer ID (TIN) (optional)": as_str(tin),
        "Eligible for 1099 (optional)": as_str(eligible_1099),
        "Rental Owner Home phone (optional)": as_str(o_home),
        "Rental Owner Work phone (optional)": as_str(o_work),
        "Rental Owner Mobile (optional)": as_str(o_mobile),
        "Rental Owner Fax (optional)": as_str(o_fax),
        "Rental Owner Date of birth (optional)": as_str(o_dob),
        "Comments (optional)": as_str(owner_comments),
        "Rental Owner Agreement start date (optional)": as_str(owner_agreement_start),
        "Rental Owner Agreement end date (optional)": as_str(owner_agreement_end),
    }


In [ ]:
print("Fetching bank accounts...")
bank_accounts = fetch_bank_accounts()
bank_accounts_by_id = {
    as_str(pick(a, "id", "accountId")): a
    for a in bank_accounts
    if pick(a, "id", "accountId")
}

print("Fetching buildings...")
buildings = fetch_buildings()
rows: List[Dict[str, Any]] = []

for b in buildings:
    b_id = as_str(pick(b, "id", "buildingId", "propertyId"))
    if not b_id:
        continue

    units = fetch_units_for_building(b_id)
    owners = fetch_owners_for_building(b_id)
    owner = owners[0] if owners else None

    for u in units:
        u_id = as_str(pick(u, "id", "unitId"))
        if not u_id:
            continue

        leases = fetch_leases_for_unit(u_id)
        lease = None
        if leases:
            active = [L for L in leases if pick(L, "status", "leaseStatus") in (None, "ACTIVE", "Active")]
            lease = active[0] if active else leases[0]

        tenants = fetch_tenants_for_lease(as_str(pick(lease or {}, "id", "leaseId"))) if lease else []
        tenant = tenants[0] if tenants else None

        rows.append(map_row(b, u, lease, tenant, owner, bank_accounts_by_id))

df = pd.DataFrame(rows, columns=COLUMNS)
df.head(10)

Fetching bank accounts...
Fetching buildings...


In [ ]:
OUT_PATH = "propertyware_export.tsv"
df.to_csv(OUT_PATH, sep="\t", index=False)
print(f"Wrote {len(df)} rows to {OUT_PATH}")
